In [17]:
# encoding=utf8
# import sys
# reload(sys)
# sys.setdefaultencoding('utf8')

import pandas as pd
import cx_Oracle
import os
import concurrent.futures

import requests
from lxml import etree
from time import time, ctime

db=cx_Oracle.connect("soyoung", "soyoung", "my.uconn.science:1521/xe")
print db.version
conn=db.cursor()

11.2.0.2.0


In [18]:
path = 'bidder_pages/'
down = os.listdir(path)
len(down)

68229

In [19]:
query="select * from bidder_compare"

In [20]:
cp = pd.read_sql(con=db, sql=query)
print cp.shape
cp.head()

(68227, 2)


,ID,URL
0,63,paddyshyam.html
1,215,revitupmedia.html
2,220,AnsonSequeira.html
3,224,Tooner.html
4,274,sakshiinfosys.html


In [21]:
# cp['UserName']=cp['URL'].str[:-5]
# cp.head()

In [22]:
df = pd.DataFrame(down, columns=['URL'])
df.head()

,URL
0,Ukrabel.html
1,observer.html
2,Ixintro.html
3,Glukodrom.html
4,blugraphic.html


In [23]:
def single(url):
    with open(path+url,'r') as fl:
        try:
            text=fl.read()
            test=etree.HTML(text)        
            name = test.xpath('//h1[@class="profile-intro-username"]/text()')[0].strip()
        except:
#             print 'X',
            name = "not exists"
    return name

In [24]:
%%time
n=0
coll=[]
for i in down:
    n+=1
    coll.append([i, single(i)])
    if n%1000==0:
        print n,
len(coll)    

1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 24000 25000 26000 27000 28000 29000 30000 31000 32000 33000 34000 35000 36000 37000 38000 39000 40000 41000 42000 43000 44000 45000 46000 47000 48000 49000 50000 51000 52000 53000 54000 55000 56000 57000 58000 59000 60000 61000 62000 63000 64000 65000 66000 67000 68000CPU times: user 3min 58s, sys: 3.34 s, total: 4min 1s
Wall time: 4min 21s



In [27]:
coll[:10]

[['Ukrabel.html', 'Ukrabel'],
 ['observer.html', 'Inicsol'],
 ['Ixintro.html', 'Ixintro'],
 ['Glukodrom.html', 'Sergey'],
 ['blugraphic.html', 'blugraphic'],
 ['RichardWeiss.html', 'RichardWeiss'],
 ['spaprikar.html', 'spaprikar'],
 ['p30patel.html', 'p30patel'],
 ['mcabezas.html', 'mcabezas'],
 ['jdbkrihn123.html', 'John Inso']]

In [35]:
final = pd.DataFrame(coll, columns=['URL','Name'])
final['UserName']=final['URL'].str[:-5]
print final.shape
final.head()

(68229, 3)


,URL,Name,UserName
0,Ukrabel.html,Ukrabel,Ukrabel
1,observer.html,Inicsol,observer
2,Ixintro.html,Ixintro,Ixintro
3,Glukodrom.html,Sergey,Glukodrom
4,blugraphic.html,blugraphic,blugraphic


In [29]:
final['Name'].value_counts().head()

not exists    13286
                  8
Karthik           4
Alex              4
Jay               4
Name: Name, dtype: int64

In [30]:
f1 = final[final['Name']!='not exists']
f1.head()

,URL,Name,UserName
0,Ukrabel.html,Ukrabel,Ukrabel
1,observer.html,Inicsol,observer
2,Ixintro.html,Ixintro,Ixintro
3,Glukodrom.html,Sergey,Glukodrom
4,blugraphic.html,blugraphic,blugraphic


In [46]:
final['Name'].value_counts()

not exists              13286
                            8
Karthik                     4
Alex                        4
Jay                         4
Sam                         4
.                           3
Project Manager             3
Paul                        3
George                      3
Vladimir                    3
Developer                   2
Nick                        2
Fatima                      2
jana7                       2
Digital Marketing           2
Pankaj Kumar                2
Marcus                      2
Wordpress Developer         2
manar                       2
Prashant                    2
Adrian                      2
Deepak                      2
Leo                         2
Abhishek                    2
Vishal Shah                 2
Shahzad                     2
Balaji                      2
melissa                     2
Freelancer                  2
                        ...  
LegalMind                   1
srichekritha                1
comastudio

In [31]:
f1[f1['Name']!=f1['UserName']].shape

(5586, 3)

In [32]:
f2 = f1[f1['Name']!=f1['UserName']]
f2.head()

,URL,Name,UserName
1,observer.html,Inicsol,observer
3,Glukodrom.html,Sergey,Glukodrom
9,jdbkrihn123.html,John Inso,jdbkrihn123
24,fahimnewaz.html,Fahim Newaz,fahimnewaz
56,manusolutions.html,WebInfozen,manusolutions


In [33]:
cp.head()

,ID,URL
0,63,paddyshyam.html
1,215,revitupmedia.html
2,220,AnsonSequeira.html
3,224,Tooner.html
4,274,sakshiinfosys.html


In [ ]:
cp['URL']

In [47]:
final3 = cp.set_index('URL').join(final.set_index('URL'), how='inner').reset_index()
final3.shape

(68227, 4)

In [48]:
final3.head()

,URL,ID,Name,UserName
0,paddyshyam.html,63,paddyshyam,paddyshyam
1,revitupmedia.html,215,revitupmedia,revitupmedia
2,AnsonSequeira.html,220,AnsonSequeira,AnsonSequeira
3,Tooner.html,224,Yury,Tooner
4,sakshiinfosys.html,274,Sakshi Infoway Pvt Ltd,sakshiinfosys


In [53]:
final5 = cp.set_index('URL').join(f1.set_index('URL'), how='inner').reset_index()
final5.shape

(54941, 4)

In [50]:
final3.to_csv('bidder_Name_full.csv', index=False, encoding='utf-8')

In [43]:
final2 = cp.set_index('URL').join(f2.set_index('URL'), how='inner').reset_index()
final2.shape

(5585, 4)

In [44]:
final2.head()

,URL,ID,Name,UserName
0,Tooner.html,224,Yury,Tooner
1,sakshiinfosys.html,274,Sakshi Infoway Pvt Ltd,sakshiinfosys
2,webdeem.html,470,Khurram Sikandar,webdeem
3,kalart.html,1107,Kaleidoscope Art,kalart
4,kalion.html,1535,Vadim Kalion,kalion


In [51]:
final2.to_csv('bidder_Name_diff.csv', index=False, encoding='utf-8')

In [52]:
final2

,URL,ID,Name,UserName
0,Tooner.html,224,Yury,Tooner
1,sakshiinfosys.html,274,Sakshi Infoway Pvt Ltd,sakshiinfosys
2,webdeem.html,470,Khurram Sikandar,webdeem
3,kalart.html,1107,Kaleidoscope Art,kalart
4,kalion.html,1535,Vadim Kalion,kalion
5,look.html,2793,Sen,look
6,suneva.html,2875,Suneva Webcasting LLC,suneva
7,netdevbiz.html,4193,Infoware Solutions,netdevbiz
8,jmawebco.html,4348,Jaime Cruz,jmawebco
9,an80smusicguy.html,4547,EMH Graphics,an80smusicguy
